In [1]:
%uv pip install 'transformers[torch]' 

Using Python 3.10.10 environment at: /teamspace/studios/this_studio/.venv
Audited 1 package in 82ms
Note: you may need to restart the kernel to use updated packages.


In [2]:
%uv pip install torchinfo 

Using Python 3.10.10 environment at: /teamspace/studios/this_studio/.venv
Audited 1 package in 2ms
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import transformers

print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)

/teamspace/studios/this_studio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.6.0+cu124
Transformers version: 4.51.0


In [4]:
from torchinfo import summary
from transformers import GPT2Tokenizer, GPT2Model

model_hf = GPT2Model.from_pretrained('gpt2')

weights = model_hf.state_dict()

summary(model_hf)


Layer (type:depth-idx)                        Param #
GPT2Model                                     --
├─Embedding: 1-1                              38,597,376
├─Embedding: 1-2                              786,432
├─Dropout: 1-3                                --
├─ModuleList: 1-4                             --
│    └─GPT2Block: 2-1                         --
│    │    └─LayerNorm: 3-1                    1,536
│    │    └─GPT2Attention: 3-2                2,362,368
│    │    └─LayerNorm: 3-3                    1,536
│    │    └─GPT2MLP: 3-4                      4,722,432
│    └─GPT2Block: 2-2                         --
│    │    └─LayerNorm: 3-5                    1,536
│    │    └─GPT2Attention: 3-6                2,362,368
│    │    └─LayerNorm: 3-7                    1,536
│    │    └─GPT2MLP: 3-8                      4,722,432
│    └─GPT2Block: 2-3                         --
│    │    └─LayerNorm: 3-9                    1,536
│    │    └─GPT2Attention: 3-10               2,362,368
│

In [5]:
for key, shape in model_hf.state_dict().items():
    
    print(key, shape.shape)


wte.weight torch.Size([50257, 768])
wpe.weight torch.Size([1024, 768])
h.0.ln_1.weight torch.Size([768])
h.0.ln_1.bias torch.Size([768])
h.0.attn.c_attn.weight torch.Size([768, 2304])
h.0.attn.c_attn.bias torch.Size([2304])
h.0.attn.c_proj.weight torch.Size([768, 768])
h.0.attn.c_proj.bias torch.Size([768])
h.0.ln_2.weight torch.Size([768])
h.0.ln_2.bias torch.Size([768])
h.0.mlp.c_fc.weight torch.Size([768, 3072])
h.0.mlp.c_fc.bias torch.Size([3072])
h.0.mlp.c_proj.weight torch.Size([3072, 768])
h.0.mlp.c_proj.bias torch.Size([768])
h.1.ln_1.weight torch.Size([768])
h.1.ln_1.bias torch.Size([768])
h.1.attn.c_attn.weight torch.Size([768, 2304])
h.1.attn.c_attn.bias torch.Size([2304])
h.1.attn.c_proj.weight torch.Size([768, 768])
h.1.attn.c_proj.bias torch.Size([768])
h.1.ln_2.weight torch.Size([768])
h.1.ln_2.bias torch.Size([768])
h.1.mlp.c_fc.weight torch.Size([768, 3072])
h.1.mlp.c_fc.bias torch.Size([3072])
h.1.mlp.c_proj.weight torch.Size([3072, 768])
h.1.mlp.c_proj.bias torch.Siz

In [6]:
weights['wpe.weight'] , weights['wpe.weight'].shape

(tensor([[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
           2.8267e-02,  5.4490e-02],
         [ 2.3959e-02, -5.3792e-02, -9.4879e-02,  ...,  3.4170e-02,
           1.0172e-02, -1.5573e-04],
         [ 4.2161e-03, -8.4764e-02,  5.4515e-02,  ...,  1.9745e-02,
           1.9325e-02, -2.1424e-02],
         ...,
         [-1.7987e-03,  1.6052e-03, -5.5103e-02,  ...,  1.3617e-02,
          -7.1805e-03,  3.7552e-03],
         [ 3.2105e-03,  1.5501e-03, -4.8944e-02,  ...,  2.0725e-02,
          -1.1838e-02, -5.5683e-04],
         [ 2.6610e-04,  3.0272e-03, -1.7086e-03,  ..., -4.6506e-03,
          -2.3541e-03, -5.7855e-03]]),
 torch.Size([1024, 768]))

In [7]:
dict({2:3})



{2: 3}

#### Working with Tinyshakespeare dataset

In [8]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r') as f:
    text = f.read()

print(text[:100])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
tokens = encoding.encode(text)
print(tokens[:100])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198, 1639, 389, 477, 12939, 2138, 284, 4656, 621, 284, 1145, 680, 30, 198, 198, 3237, 25, 198, 4965, 5634, 13, 12939, 13, 198, 198, 5962, 22307, 25, 198, 5962, 11, 345, 760, 327, 1872, 385, 1526, 28599, 318, 4039, 4472, 284, 262, 661, 13, 198, 198, 3237, 25, 198, 1135, 760, 470, 11, 356, 760, 470, 13, 198, 198, 5962, 22307, 25, 198, 5756, 514, 1494, 683, 11, 290, 356]


In [10]:
#we need to put this in a way of (Batch, sequence_length)
import torch

buff = torch.tensor(tokens[:24])
x = buff.view(4,6)

In [11]:
#getting the targets
buff = torch.tensor(tokens[:24+1])
x = buff[:-1].view(4,6)
y = buff[1:].view(4,6)
x,y #now we have labels for each one

(tensor([[ 5962, 22307,    25,   198,  8421,   356],
         [ 5120,   597,  2252,    11,  3285,   502],
         [ 2740,    13,   198,   198,  3237,    25],
         [  198,  5248,   461,    11,  2740,    13]]),
 tensor([[22307,    25,   198,  8421,   356,  5120],
         [  597,  2252,    11,  3285,   502,  2740],
         [   13,   198,   198,  3237,    25,   198],
         [ 5248,   461,    11,  2740,    13,   198]]))

In [20]:
True if 'lm_head' in weights.keys() else False

False

In [18]:
weights['ln_f.weight'].shape, weights['wte.weight'].shape

(torch.Size([768]), torch.Size([50257, 768]))

In [1]:
#This eg shows how adding of random weights can lead to the overflow of values
#why we need the layer normalization to control it
# This example shows visually how repeated additions can cause a vector's "noise" or variability to grow significantly, making normalization crucial in deep neural networks.


import torch
x = torch.zeros(768)

for i in range(100): #100 layers
    x += torch.randn(768) 

print(x.std()) #how spreaded your values are

tensor(9.5927)


This shows that as we keep on adding the values . Our values get keep on growing due to which it becomes unstable
to train . 

-> Thats when we use layer normalization to stabilize the values

In [2]:
#so above you could see that the values were exploded
#buf if we normalize it with standard deviation


# A. Standard Deviation
x = torch.zeros(768)
for i in range(100): #100 layers
    x += torch.randn(768) * 0.02 

print(x.std()) #how spreaded your values are
# (✓100 * 0.02) = 0.2
#so now the values will be more controlled


# B. Residual Connection (skip connections)
x = torch.zeros(768)
for _ in range(100):
    x = x + (torch.randn(768) * 0.02)  # Residual: x = x + F(x)
print(x.std())  # Output: ~0.2 (controlled)


# C. Layer Normalization
x = torch.zeros(768)
for _ in range(100):
    x = x + torch.randn(768) * 0.02
    x = (x - x.mean()) / (x.std() + 1e-5)  # Normalize
print(x.std())  # Output: ~1.0 (forced to unit variance)

tensor(0.2071)
tensor(0.2085)
tensor(1.0000)


In [3]:
#we can control the scaling either this way

x = torch.zeros(768)
n = 100
for i in range(100): #100 layers
    x += n ** -0.5 * torch.randn(768)

print(x.std()) #how spreaded your values are

tensor(0.9559)


Let's explain this clearly, step-by-step, to understand why scaling by \(1/\sqrt{n}\) works:

---

### **Step 1: Understanding without scaling**

Suppose you add random noise to a vector repeatedly (without scaling):

```python
x = torch.zeros(768)
for i in range(n):
    x += torch.randn(768)
```

**What happens?**  
Each random vector you add has a standard deviation of about **1**.  
When you add these vectors repeatedly, the variances **add up**.

- Variance after 1 addition: **1**
- Variance after 2 additions: **2**
- Variance after \( n \) additions: **n**

**Standard deviation** = √(variance)  
Thus, after adding random noise \( n \) times, the standard deviation grows to:

\[
\sqrt{n}
\]

---

### **Step 2: Why scaling by \(1/\sqrt{n}\)?**

You scaled each addition like this:

```python
x += torch.randn(768) / sqrt(n)
```

Now, each added vector has standard deviation:

\[
\frac{1}{\sqrt{n}}
\]

**Let's find the variance clearly:**  
- Each scaled addition now has **variance** \( (1/\sqrt{n})^2 = 1/n \).
- After adding these scaled vectors \( n \) times, the total variance is:

\[
\text{variance after n additions} = n \times \frac{1}{n} = 1
\]

Thus, the variance remains constant (**1**) even after \( n \) additions.

---

### **Step 3: The intuitive reason why it works**

- **Without scaling:** Each addition makes the spread (standard deviation) bigger, because you keep adding the same-sized noise.
- **With scaling:** You're adding smaller noise each time, exactly chosen so that the total spread stays the same.

You choose exactly **\(1/\sqrt{n}\)** because variance grows linearly (by addition), and standard deviation grows by the square root. Thus, dividing by **√n** precisely compensates for the growth that would otherwise happen.

---

### **Simple summary:**
- Without scaling → The spread grows as you add more random vectors.
- With scaling by \(1/\sqrt{n}\) → The spread remains stable, at about 1.

This scaling trick is crucial to keep numerical stability in neural network computations, ensuring consistent behavior across layers.

In [17]:
100 ** -0.5

0.1